In [1]:
# scripts/exploracao_merge_dataset.ipynb

# ==========================
#   EDA - MERGE DATASET
# ==========================
# (1) Setup inicial
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.loader import load_merge_master

# Caminho para o master_metadata que deseja explorar
MASTER_PATH = '../metadata/master_metadata_audio_balanced.csv'  # ajuste conforme a modalidade

# (2) Carregar os dados (usando o loader.py)
df = load_merge_master(MASTER_PATH)
print(f"Total de músicas no dataset: {len(df)}")
display(df.head())

# (3) Resumo dos campos
print("\nCampos disponíveis:", df.columns.tolist())
print("\nInformações do DataFrame:")
display(df.info())
display(df.describe(include='all'))

# (4) Contagem por Split (se existir)
for split_col in ['split_70_15_15', 'split_40_30_30']:
    if split_col in df.columns:
        display(df[split_col].value_counts())
        df[split_col].value_counts().plot.pie(autopct='%1.1f%%', title=f'Proporção de {split_col}')
        plt.ylabel('')
        plt.show()

# (5) Distribuição por Quadrante
if 'quadrant' in df.columns:
    sns.countplot(x='quadrant', data=df)
    plt.title('Distribuição por Quadrante Emocional')
    plt.xlabel('Quadrante')
    plt.ylabel('Nº de Músicas')
    plt.show()

# (6) Distribuição por Ano (year ou ActualYear)
for year_col in ['year', 'ActualYear', 'Year']:
    if year_col in df.columns:
        plt.figure(figsize=(10,4))
        sns.histplot(df[year_col].dropna(), bins=20, kde=True)
        plt.title(f'Distribuição por Ano ({year_col})')
        plt.xlabel('Ano')
        plt.ylabel('Nº de Músicas')
        plt.show()
        break

# (7) Distribuição por Gêneros (top 15)
for gcol in ['genres', 'Genres']:
    if gcol in df.columns:
        from collections import Counter
        gen = df[gcol].dropna().str.cat(sep=';').split(';')
        gen = [g.strip().lower() for g in gen if g.strip()]
        top_gen = Counter(gen).most_common(15)
        genres_df = pd.DataFrame(top_gen, columns=['Gênero', 'Frequência'])
        plt.figure(figsize=(10,5))
        sns.barplot(y='Gênero', x='Frequência', data=genres_df)
        plt.title('Top 15 Gêneros Musicais')
        plt.show()
        break

# (8) Top artistas e músicas
for acol in ['artist', 'Artist']:
    if acol in df.columns:
        top_artists = df[acol].value_counts().head(10)
        print('\nTop 10 artistas:')
        display(top_artists)
        top_artists.plot.bar(title='Top 10 Artistas')
        plt.xlabel('Artista')
        plt.ylabel('Nº de músicas')
        plt.show()
        break

for tcol in ['title', 'Title']:
    if tcol in df.columns:
        print('\nExemplo de músicas:')
        display(df[[tcol]].sample(10))
        break

# (9) Histograma de arousal e valence (se existirem)
for acol in ['arousal', 'valence']:
    if acol in df.columns:
        sns.histplot(df[acol], bins=20, kde=True)
        plt.title(f'Distribuição de {acol}')
        plt.xlabel(acol.capitalize())
        plt.ylabel('Nº de Músicas')
        plt.show()

# (10) Heatmap arousal x valence
if all(c in df.columns for c in ['arousal', 'valence']):
    plt.figure(figsize=(6,5))
    sns.scatterplot(data=df, x='valence', y='arousal', hue='quadrant' if 'quadrant' in df.columns else None, alpha=0.5)
    plt.title('Distribuição Arousal x Valence')
    plt.show()

# (11) Análise de dados faltantes
plt.figure(figsize=(12,5))
sns.heatmap(df.isnull(), cbar=False)
plt.title('Mapa de Dados Faltantes')
plt.show()

print("EDA concluída! Salve gráficos ou copie tabelas para o relatório final.")



ModuleNotFoundError: No module named 'scripts.loader'

In [ ]:
# =============================
#   Exploratory Data Analysis MERGE Dataset
# =============================

# 1. Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.loader import load_merge_master

# 2. Configuração e seleção do master_metadata
master_path = 'metadata/master_metadata_audio_balanced.csv'  # Troque pelo master que quiser
df = pd.read_csv(master_path)
print(f"Shape: {df.shape}")
print("Colunas:", df.columns.tolist())

# 3. Amostra dos dados
display(df.head(10))

# 4. Checagem básica
print(df.info())
print("\nDados ausentes por coluna:")
print(df.isnull().sum())

# 5. Contagem por splits (train/val/test)
for split_col in ['split_70_15_15', 'split_40_30_30']:
    if split_col in df.columns:
        print(f"\nDistribuição do split {split_col}:")
        print(df[split_col].value_counts())
        sns.countplot(data=df, x=split_col)
        plt.title(f'Distribuição dos splits: {split_col}')
        plt.show()

# 6. Distribuição de quadrantes
if 'quadrant' in df.columns:
    print("\nDistribuição por quadrante:")
    print(df['quadrant'].value_counts())
    sns.countplot(data=df, x='quadrant')
    plt.title('Distribuição de Quadrantes Emocionais')
    plt.show()

# 7. Distribuição de arousal/valence
if 'arousal' in df.columns and 'valence' in df.columns:
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    sns.histplot(df['arousal'], bins=20, kde=True, ax=ax[0])
    ax[0].set_title('Distribuição: Arousal')
    sns.histplot(df['valence'], bins=20, kde=True, ax=ax[1])
    ax[1].set_title('Distribuição: Valence')
    plt.show()

    # Dispersão arousal vs valence por quadrante
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=df, x='valence', y='arousal', hue='quadrant', palette='tab10')
    plt.title('Arousal vs Valence (colorido por quadrante)')
    plt.show()

# 8. Distribuição de anos (quando houver coluna year/actualyear)
for ycol in ['year', 'Year', 'ActualYear']:
    if ycol in df.columns:
        print(f"\nDistribuição por ano: {ycol}")
        print(df[ycol].value_counts().sort_index())
        plt.figure(figsize=(12,4))
        sns.countplot(data=df, x=ycol, order=sorted(df[ycol].dropna().unique()))
        plt.xticks(rotation=90)
        plt.title(f'Distribuição por Ano ({ycol})')
        plt.show()
        break

# 9. Top artistas/títulos/gêneros (quando disponível)
for acol in ['artist', 'Artist']:
    if acol in df.columns:
        print("\nTop 15 artistas:")
        print(df[acol].value_counts().head(15))
        sns.countplot(y=df[acol], order=df[acol].value_counts().head(10).index)
        plt.title('Top 10 Artistas')
        plt.show()
        break

for tcol in ['title', 'Title']:
    if tcol in df.columns:
        print("\nTop 15 músicas:")
        print(df[tcol].value_counts().head(15))
        break

for gcol in ['genres', 'Genres']:
    if gcol in df.columns:
        from collections import Counter
        all_genres = df[gcol].dropna().astype(str).str.cat(sep=';').split(';')
        genres_count = Counter([g.strip().lower() for g in all_genres if g.strip() != ''])
        print("\nTop 20 gêneros:")
        for g, v in genres_count.most_common(20):
            print(f"{g}: {v}")
        top_genres = [g for g, _ in genres_count.most_common(10)]
        plt.figure(figsize=(8,5))
        sns.barplot(y=top_genres, x=[genres_count[g] for g in top_genres], orient='h')
        plt.title('Top 10 Gêneros')
        plt.xlabel('Frequência')
        plt.ylabel('Gênero')
        plt.show()
        break

# 10. Distribuição por duração (se existir)
for dcol in ['duration', 'Duration']:
    if dcol in df.columns:
        plt.figure(figsize=(8,4))
        sns.histplot(df[dcol].dropna(), bins=30, kde=True)
        plt.title('Distribuição de Duração das Músicas')
        plt.xlabel('Duração (s)')
        plt.show()
        break

# 11. Relação entre splits e quadrantes
if 'quadrant' in df.columns:
    for split_col in ['split_70_15_15', 'split_40_30_30']:
        if split_col in df.columns:
            print(f"\nCruzamento {split_col} vs Quadrant:")
            cross = pd.crosstab(df[split_col], df['quadrant'])
            print(cross)
            sns.heatmap(cross, annot=True, fmt='d')
            plt.title(f'{split_col} vs Quadrant')
            plt.show()

# 12. Relação entre gêneros e quadrantes (opcional)
if 'quadrant' in df.columns and 'genres' in df.columns:
    print("\nExemplo: principais gêneros por quadrante:")
    quadrants = sorted(df['quadrant'].unique())
    for q in quadrants:
        sub = df[df['quadrant']==q]
        all_genres = sub['genres'].dropna().astype(str).str.cat(sep=';').split(';')
        genres_count = Counter([g.strip().lower() for g in all_genres if g.strip() != ''])
        print(f"Quadrant {q}: {[g for g, v in genres_count.most_common(5)]}")

# 13. Conclusão automática (resuma principais insights ou exporte os gráficos)
print("\n== FIM DA EDA ==")
